# Técnicas Computacionales Avanzadas de Hidro/Aero-Dinámica en Ingeniería y Astrofísica

## Elementos Finitos: Metodo Galerkin Discontinuo

**MSc Erick Urquilla, Universidad de Tennessee, Knoxville, USA**

Resolveremos las ecuaciones que gobiernan la dinámica de fluidos poco profundos incompresibles. El fluido está limitado inferiormente y a los costados por superficies rígidas y superiormente sin restricción. Estas ecuaciones se derivan de las ecuaciones de Navier-Stokes cuando el dominio espacial horizontal del fluido excede significativamente al vertical, manteniendo el equilibrio hidrostático y densidad constante:

$$
\frac{\partial h}{\partial t} + \frac{\partial}{\partial x}\left(hu\right) = 0
$$
$$
\frac{\partial}{\partial t}\left(hu\right) + \frac{\partial}{\partial x}\left(hu^2 + \frac{gh^2}{2}\right) = 0
$$

Aquí, $h$ es la altura del fluido, $u$ es la velocidad horizontal y $g$ es la aceleración de la gravedad. El movimiento del fluido está dictado exclusivamente por las ecuaciones de conservación de masa y momento.

### Importaremos los paquetes necesarios

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import time
import os
import glob
import imageio.v2 as imageio
import sys
from pathlib import Path

### Importando los scripts con el código de Galerkin discontinuo

In [ ]:
aqui_camino = Path.cwd()
cdg_fuente_camino = (aqui_camino / '..' / 'cdg_fuente').resolve()
sys.path.append(str(cdg_fuente_camino))

import bases
import galerkin_discontinuo 
import paso_de_tiempo
import graficos

### Parámetros de entrada

| Nombre de la variable | Unidades | Descripción |
|-----------------------|----------|-------------|
| **x_inicial**         | m        | Coordenada inicial del dominio espacial donde comienza la simulación. |
| **x_final**           | m        | Coordenada final del dominio espacial.|
| **N_elementos**       | —        | Número de elementos finitos en los que se subdivide el dominio espacial. Afecta la resolución espacial. |
| **N_nodos**           | —        | Número de nodos (o puntos de interpolación) dentro de cada elemento. Para simplicidad se asume `N_nodos ≥ 2`. |
| **n_pasos**           | —        | Número de pasos de tiempo que se ejecutarán en la simulación. Determina la resolución temporal. |
| **t_total**           | s        | Tiempo total de la simulación. |
| **n_nodos_cuadratura_gauss** | — | Número de nodos (puntos) utilizados en la cuadratura de Gauss para integrar numéricamente las funciones dentro de cada elemento. Un mayor número incrementa la precisión de la integración. |


In [ ]:
# Dominio espacial de la simulacion 
x_inicial = 0 # (m) cooordenda inicial del dominio
x_final = 10 # (m) coordenada final del dominio

# Parametros del metodo de elementos finitos
N_elementos = 6 # numero de elementos
N_nodos = 4 # numero de nodos por elemento (por simplicidad solo consideramos N_nodos >= 2)

# Dominio temporal 
n_pasos = 100 # numero de pasos temporales
t_total = 1 # (s) tiempo final

# Integracion numerica con cuadratura de Gauss
n_nodos_cuadratura_gauss = 20

## Generacion de malla

La generación de una malla es fundamental en el método de elementos finitos. La malla divide el dominio espacial en elementos finitos más pequeños, lo que permite aproximar la solución de la ecuación diferencial en cada uno de estos elementos. En el método Galerkin Discontinuo, cada elemento puede tener su propia solución polinómica, lo que permite capturar discontinuidades y variaciones abruptas en las variables del fluido.

**Ejercicio 1:** Dadas las coordenadas inicial `x_inicial`, coordenada final `x_final`, el número de elementos `N_elementos` y el número de nodos por elemento `N_nodos`. Codifica una malla cartesiana unidimensional con las coordenadas de los nodos interiores de cada elemento (elementos y nodos igualmente espaciados en una malla regular). El numpy array resultado del ejercicio debe llamarse `malla`. La forma del array `malla` debe ser `(N_elementos, N_nodos)`. Es decir, el componente `malla[i,j]` representa la coordenada en `x` del nodo `j` del elemento `i`. 

Como ejemplo, observa la siguiente imagen de una `malla` con `N_elementos=3` y `N_nodos=4`. El arreglo `malla` contiene las posiciones de los nodos interiores de cada elemento.

![Malla](../docs/malla.png)

In [ ]:
# Generando malla
longitud_elemento = (x_final - x_inicial) / N_elementos

# Generar coordenadas de los elementos y nodos en el espacio físico
malla = np.zeros((N_elementos, N_nodos)) # nunpy array que almacenara las coordenadas de los nodos de cada elemento

#-----------------------------------------------------------------------------------------
# Escribe tu solucion al ejercicio 1 a continuacion ...

malla = np.array([np.linspace(x_inicial + i * (x_final - x_inicial) / N_elementos, x_inicial + (i + 1) * (x_final - x_inicial) / N_elementos, N_nodos) for i in range(N_elementos)])      

#-----------------------------------------------------------------------------------------


## Condiciones Iniciales

Las condiciones iniciales del fluido están dadas por:

$$h_0 = 1 + 0.1 e^{-(x-5)^2}$$
$$u_0 = 0$$

Las fronteras del fluido están ubicadas en $x = 0$ y $x = 10$ metros.

**Ejercicio 2:** Crea dos arrays de numpy que contengan las condiciones iniciales:

1. El primer array debe llamarse `h` y tener la forma `(N_elementos, N_nodos)`. `h[i,j]` representará la altura en metros del fluido en el nodo $j$ del elemento $i$.
2. El segundo array debe llamarse `u` y tener la forma `(N_elementos, N_nodos)`. `u[i,j]` representará la velocidad en metros por segundo del fluido en el nodo $j$ del elemento $i$.

In [ ]:
# Generando condiciones iniciales
h = np.zeros((N_elementos, N_nodos)) # Height (cm)
u = np.zeros((N_elementos, N_nodos)) # Velocity (cm/s)

#-----------------------------------------------------------------------------------------
# Escribe tu solucion al ejercicio 2 a continuacion ...
h = 1.0 + 0.1 * np.exp( - ( malla - 5.0 )**2 ) # Height (cm)
u = malla * 0.0 # Velocity (cm/s)
#-----------------------------------------------------------------------------------------

### Graficando la solución

In [ ]:
graficos.plot_simulation(malla, h, u, N_elementos, 0, 0, display=True, save=False)